In [5]:
import logging

import pickle

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from torch.distributions import constraints
from torch import nn
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
import seaborn as sns
from pyro import poutine
from sklearn import metrics

In [6]:
pyro.set_rng_seed(10)

In [7]:

with open('data_all.pickle', 'rb') as handle:
    data = pickle.load(handle)
print(data.shape)

nan_mask = np.isnan(data) #when calculating the train/test set to "nan" all the examples that are for testing so that you do not train on them 
print(torch.from_numpy(nan_mask) )

with open('data_train.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
print(data2.shape)

nan_mask = np.isnan(data2) #when calculating the train/test set to "nan" all the examples that are for testing so that you do not train on them 
print(torch.from_numpy(nan_mask) )


(1127, 5237)
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
(1127, 5237)
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])


In [8]:
class PMF_NB_with_drug_varying_alpha(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, data,train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = data.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None
        self.returned = None
        self.predictive_svi  = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()
        
        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
        self.bias = self.data.mean()


    def model(self, train, mask):
        a = 50

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

        with drug_plate: 
            UA = pyro.sample("UA", dist.Gamma(self.alpha_u, self.beta_u).expand([self.dim]).to_event(1))
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
            exposure = pyro.sample("exposure", dist.Gamma(a,a))
        
        with sideeffect_plate:
            VA = pyro.sample("VA", dist.Gamma(self.alpha_v, self.beta_v).expand([self.dim]).to_event(1))
            #possibly add intercepts VA_int = pyro.sample("VA", dist.Normal(0., 1.).to_event(1))
       
        u2_plate = pyro.plate("u2_plate", self.n, dim=-2)

        with sideeffect_plate, u2_plate: 
            with pyro.poutine.mask(mask=mask):
             Y = pyro.sample("target", dist.Poisson(exposure[:, np.newaxis]*(UA@VA.T )), obs=train ) 
             return Y
        

    def guide(self, train=None, mask=None):

        d_alpha = pyro.param('d_alpha', torch.ones(self.n,self.dim), constraint=constraints.positive)#*self.user_mean)
        d_beta = pyro.param('d_beta', 0.5*torch.ones(self.n,self.dim), constraint=constraints.positive)
        exp_alpha = pyro.param('exp_alpha', 10*torch.ones(self.n), constraint=constraints.positive)

        s_alpha = pyro.param('s_alpha', torch.ones(self.m,self.dim), constraint=constraints.positive)#*self.item_mean)
        s_beta = pyro.param('s_beta', 0.5*torch.ones(self.m,self.dim), constraint=constraints.positive)
        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

        with drug_plate: 
            UA = pyro.sample("UA", dist.Gamma(d_alpha, d_beta).to_event(1))
            exposure = pyro.sample("exposure", dist.Gamma(exp_alpha,exp_alpha))
        with sideeffect_plate: 
            VA = pyro.sample("VA", dist.Gamma(s_alpha, s_beta).to_event(1))
    
    def train_SVI(self,train,mask, nsteps=500, lr = 0.05, lrd = 1, verbose=True):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float(), mask)
            losses.append(elbo)
            if(verbose):
                if step % 10 == 0:
                    print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        unmasked =torch.ones((self.n,self.m), dtype=torch.bool)
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)(None , unmasked)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["exposure"].numpy()
        self.predictive_svi = predictive_svi
        table = predictive_svi["target"].numpy()
        self.predictive_svi = predictive_svi
        self.returned = table
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)

  
        self.predictions = mc_table
        
    def rmse(self,test,masked,h):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < h] = low
        test_data[test_data >= h] = high
        size = masked.sum()
        predictions = self.predictions
        predictions[predictions < h] = low
        predictions[predictions >= h] = high
        print(predictions.shape)
        print(masked.shape)
        sqerror = abs(test_data[masked] - predictions[masked]) ** 2  # squared error array
        mse = sqerror.sum()/size
        print("PMF  RMSE: " , np.sqrt(mse))
        fpr, tpr, thresholds = metrics.roc_curve(test_data[masked].astype(int).flatten(),  predictions[masked].astype(int).flatten(), pos_label=1)
        metrics.auc(fpr, tpr)
        print("AUC: %.5f" % metrics.auc(fpr, tpr))
        return np.sqrt(mse) , metrics.auc(fpr, tpr)

    def get_predictions(self):
        return (self.returned,self.predictions)

    def get_predictive_svi(self):
         return (self.predictive_svi)

    
    

In [9]:

test = PMF_NB_with_drug_varying_alpha(data,data2,50)
l = test.train_SVI(data, ~torch.from_numpy(nan_mask))

Elbo loss: 1065275266.6543274
Elbo loss: 169799746.33642578
Elbo loss: 58979671.5663147
Elbo loss: 35360938.697052
Elbo loss: 31147146.61416626
Elbo loss: 28951434.214385986
Elbo loss: 27969522.804779053
Elbo loss: 26542591.08230591
Elbo loss: 24511143.21245575
Elbo loss: 22388412.926776886
Elbo loss: 20552185.680530548
Elbo loss: 19355309.5254364
Elbo loss: 18762569.479995728
Elbo loss: 17964894.622833252
Elbo loss: 17523390.178359985
Elbo loss: 17191609.505218506
Elbo loss: 16971229.588867188
Elbo loss: 16855621.918823242
Elbo loss: 16449360.611114502
Elbo loss: 16468665.700683594
Elbo loss: 16095948.288757324
Elbo loss: 15798037.846832275
Elbo loss: 16012890.302490234
Elbo loss: 15784177.328674316
Elbo loss: 15280610.742553711
Elbo loss: 15792921.825042725
Elbo loss: 15382473.724273682
Elbo loss: 15510881.202453613
Elbo loss: 15389410.067352295
Elbo loss: 15417605.04397583
Elbo loss: 15437211.991821289
Elbo loss: 15259442.893493652
Elbo loss: 15262806.54473877
Elbo loss: 14934635.09

In [10]:
test.sample_predict(300)

UA: (300, 1, 1127, 50)
exposure: (300, 1, 1127)
VA: (300, 1, 5237, 50)
target: (300, 1127, 5237)


In [7]:
print(data2[nan_mask])

[nan nan nan ... nan nan nan]


In [11]:
test.rmse(data, nan_mask,1)
#train rmse
test.rmse(data, ~nan_mask,1)

(1127, 5237)
(1127, 5237)
PMF  RMSE:  0.3560744092446151
AUC: 0.83054
(1127, 5237)
(1127, 5237)
PMF  RMSE:  0.34841491146581494
AUC: 0.83613


(0.34841491146581494, 0.8361346607902966)

In [12]:
test.rmse(data, nan_mask,4)
#train rmse
test.rmse(data, ~nan_mask,4)

(1127, 5237)
(1127, 5237)
PMF  RMSE:  0.2935781208202841
AUC: 0.50000
(1127, 5237)
(1127, 5237)
PMF  RMSE:  0.29386885556227027
AUC: 0.50000


(0.29386885556227027, 0.5)

In [13]:
Simple_PMF ={}
Simple_PMF['losses'] = l 
#Simple_PMF['predictive_score'] = d
Simple_PMF['train_rmse'] = .34841
Simple_PMF['test_rmse'] = 0.35607
Simple_PMF['AUC'] = 0.838
Simple_PMF['train_rmse_3'] = 0.2938
Simple_PMF['test_rmse_3'] = 0.2935
Simple_PMF['predictions'] = test.get_predictions()

In [14]:
with open('SVI_negative_binomial_with_prior.pickle', 'wb') as handle:
    pickle.dump(Simple_PMF, handle, protocol=pickle.HIGHEST_PROTOCOL)